### Importing Librares

In [1]:
import pandas as pd
import requests
from google.cloud import vision
import concurrent.futures
from io import BytesIO
import time

### Read data of filtered and modified accommodation galleries from csv (output of Notebook: Data_Preparation_and_ duration_ api)

In [2]:
file='csv/data_filtered_google_vision_input.csv'
new_df= pd.read_csv(file)


In [3]:
new_df

,accommodation_id,longitude,latitude,driving_duration,walking_duration,transit_duration,image_id,image,tag_name,probability,accommodation_name,dt,distance_to_bigben
0,37151070,-0.15444,51.49080,16,56,10,BZU37N77BV2F3LHYDM3TMMQOXKZMGOW5B6N6MZYAOCXCF7...,https://imgcy.trivago.com/partner-images/0e/69...,bedroom,0.9999,Passmore Street By Onefinestay,2024-03-07,3.395155
1,37151070,-0.15444,51.49080,16,56,10,65OO2YWIMBEYPYI7TULDEBYJ65W5HRUUUF3PMBYLBTS5JE...,https://imgcy.trivago.com/partner-images/f7/5c...,bedroom,0.9954,Passmore Street By Onefinestay,2024-03-07,3.395155
2,37151070,-0.15444,51.49080,16,56,10,RZRSSVMYZXVLIQT2VXQAKNO524ZEDJ65NG3JNWOZIYMV3F...,https://imgcy.trivago.com/partner-images/8e/63...,bedroom,0.9999,Passmore Street By Onefinestay,2024-03-07,3.395155
3,37151070,-0.15444,51.49080,16,56,10,Y5IZPSMKFH26KUHXOFJRYAEXKZVEL6TCHLDRNUI4LRW64A...,https://imgcy.trivago.com/partner-images/c7/51...,building,0.9996,Passmore Street By Onefinestay,2024-03-07,3.395155
4,37151070,-0.15444,51.49080,16,56,10,NUV5T3RYBEGCC4GXLJ3FA3TVCQSQJVPPMGZFPDSNAP6JMY...,https://imgcy.trivago.com/partner-images/6d/2b...,kids-area,0.9716,Passmore Street By Onefinestay,2024-03-07,3.395155
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76671,25253350,-0.12750,51.50722,4,14,10,OPRDTFXE7GRZWBINQLFHVBCBF67X2ISGDYJOHTMG5AE3QP...,https://imgcy.trivago.com/hotelier-images/73/e...,bedroom,0.9999,2 Bedroom Flat In Mayfair,2024-03-07,0.820251
76672,25253350,-0.12750,51.50722,4,14,10,RZFWVMH5TW32S67G74L5EHKDOXAAFKM45O7ERHFZ2TOZ2X...,https://imgcy.trivago.com/partner-images/8e/4b...,bedroom,0.9999,2 Bedroom Flat In Mayfair,2024-03-07,0.820251
76673,25253350,-0.12750,51.50722,4,14,10,NGQMN5KJNRMF3TERX7ZG526HV2VWHCACC7DA7KOHNUHDXY...,https://imgcy.trivago.com/partner-images/69/a0...,building,0.9999,2 Bedroom Flat In Mayfair,2024-03-07,0.820251
76674,25253350,-0.12750,51.50722,4,14,10,TSY3I2RRLJTKYU67XEIMGE4EJ72TA3APVHLRWX44OXLUJR...,https://imgcy.trivago.com/partner-images/9c/b1...,bedroom,0.9969,2 Bedroom Flat In Mayfair,2024-03-07,0.820251


### Landmark detection function

In [4]:
new_df['description'] = ''
new_df['score'] = 0.0
new_df['response_time']= 0.0
requests_per_minute_limit = 1800
seconds_per_request = 60.0

In [5]:
def detect_landmarks_uri(uri):
    """Detects landmarks in the file located in Google Cloud Storage or on the Web."""
    client = vision.ImageAnnotatorClient()
    
    try:
        # Download the image
        response = requests.get(uri)
        response.raise_for_status()
        content = response.content

        # Use the downloaded content
        image = vision.Image(content=content)
        
        start_time = time.time()
        response = client.landmark_detection(image=image)
        end_time = time.time()
        
        response_time = end_time - start_time

        if response.error.message:
            raise Exception(
                "{}\nFor more info on error messages, check: "
                "https://cloud.google.com/apis/design/errors".format(response.error.message)
            )

        landmarks = response.landmark_annotations
        if landmarks:
            # Get the top landmark and its score
            top_landmark = max(landmarks, key=lambda x: x.score)
            description = top_landmark.description
            score = top_landmark.score
        else:
            description = None  # Or "No landmarks detected"
            score = 0.0

    except requests.RequestException as e:
        print(f"Error downloading {uri}: {e}")
        return uri, None, 0.0, None
    except Exception as e:
        print(f"Error processing {uri}: {e}")
        return uri, None, 0.0, None
    
    return uri, description, score, response_time

def process_folder(folder_path):
    results = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            file_path = os.path.join(folder_path, filename)
            result = detect_landmarks_uri(file_path)
            results.append(result)
    return results


### Function for Parallel Processing

In [6]:
def process_images_parallel(df, max_workers=10):
    """Process multiple images in parallel and record response times."""
    def process_row(uri):
        try:
            uri, description, score, response_time = detect_landmarks_uri(uri)
            return uri, description, score, response_time
        except Exception as e:
            print(f"Error processing {uri}: {e}")
            return uri, "Error", 0.0, None

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(process_row, uri): uri for uri in df['image']}
        
        for future in concurrent.futures.as_completed(futures):
            uri, description, score, response_time = future.result()
            df.loc[df['image'] == uri, 'description'] = description
            df.loc[df['image'] == uri, 'score'] = score
            df.loc[df['image'] == uri, 'response_time'] = response_time

    return df

In [7]:
new_df = process_images_parallel(new_df , max_workers=10)


Error processing https://imgcy.trivago.com/hotelier-images/38/6b/814c67a6f7c520974bb58182540e7ed648dc90427deb26fc54c16b10d5b9: 400 Request payload size exceeds the limit: 41943040 bytes.
Error processing https://imgcy.trivago.com/hotelier-images/d9/1f/4be2e17ab537e718704b11fc3a420c0287a8afb7a7356e139fb7b676d658: 400 Request payload size exceeds the limit: 41943040 bytes.
Error processing https://imgcy.trivago.com/hotelier-images/68/b7/ba7679080bd9c5f54fcfb0f45a5ab9e51a662d2fcf4198d6389cbf2772f1: 400 Request payload size exceeds the limit: 41943040 bytes.
Error processing https://imgcy.trivago.com/hotelier-images/b2/f9/948fbe9e0af6b842bae3081479bb26a4ed5778e29b1eff0970d54c93a1f3: 400 Request payload size exceeds the limit: 41943040 bytes.
Error processing https://imgcy.trivago.com/hotelier-images/92/16/157d2ef06bf2523abc0d83b7e0d38c0d4ee3296acb6b68966830de2ee2f0: 400 Request payload size exceeds the limit: 41943040 bytes.
Error processing https://imgcy.trivago.com/hotelier-images/bb/80/

In [8]:
new_df

,accommodation_id,longitude,latitude,driving_duration,walking_duration,transit_duration,image_id,image,tag_name,probability,accommodation_name,dt,distance_to_bigben,description,score,response_time
0,37151070,-0.15444,51.49080,16,56,10,BZU37N77BV2F3LHYDM3TMMQOXKZMGOW5B6N6MZYAOCXCF7...,https://imgcy.trivago.com/partner-images/0e/69...,bedroom,0.9999,Passmore Street By Onefinestay,2024-03-07,3.395155,None,0.0,0.273663
1,37151070,-0.15444,51.49080,16,56,10,65OO2YWIMBEYPYI7TULDEBYJ65W5HRUUUF3PMBYLBTS5JE...,https://imgcy.trivago.com/partner-images/f7/5c...,bedroom,0.9954,Passmore Street By Onefinestay,2024-03-07,3.395155,None,0.0,0.338291
2,37151070,-0.15444,51.49080,16,56,10,RZRSSVMYZXVLIQT2VXQAKNO524ZEDJ65NG3JNWOZIYMV3F...,https://imgcy.trivago.com/partner-images/8e/63...,bedroom,0.9999,Passmore Street By Onefinestay,2024-03-07,3.395155,None,0.0,0.327386
3,37151070,-0.15444,51.49080,16,56,10,Y5IZPSMKFH26KUHXOFJRYAEXKZVEL6TCHLDRNUI4LRW64A...,https://imgcy.trivago.com/partner-images/c7/51...,building,0.9996,Passmore Street By Onefinestay,2024-03-07,3.395155,None,0.0,0.542081
4,37151070,-0.15444,51.49080,16,56,10,NUV5T3RYBEGCC4GXLJ3FA3TVCQSQJVPPMGZFPDSNAP6JMY...,https://imgcy.trivago.com/partner-images/6d/2b...,kids-area,0.9716,Passmore Street By Onefinestay,2024-03-07,3.395155,None,0.0,0.615594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76671,25253350,-0.12750,51.50722,4,14,10,OPRDTFXE7GRZWBINQLFHVBCBF67X2ISGDYJOHTMG5AE3QP...,https://imgcy.trivago.com/hotelier-images/73/e...,bedroom,0.9999,2 Bedroom Flat In Mayfair,2024-03-07,0.820251,None,0.0,0.131110
76672,25253350,-0.12750,51.50722,4,14,10,RZFWVMH5TW32S67G74L5EHKDOXAAFKM45O7ERHFZ2TOZ2X...,https://imgcy.trivago.com/partner-images/8e/4b...,bedroom,0.9999,2 Bedroom Flat In Mayfair,2024-03-07,0.820251,None,0.0,0.128994
76673,25253350,-0.12750,51.50722,4,14,10,NGQMN5KJNRMF3TERX7ZG526HV2VWHCACC7DA7KOHNUHDXY...,https://imgcy.trivago.com/partner-images/69/a0...,building,0.9999,2 Bedroom Flat In Mayfair,2024-03-07,0.820251,None,0.0,0.150750
76674,25253350,-0.12750,51.50722,4,14,10,TSY3I2RRLJTKYU67XEIMGE4EJ72TA3APVHLRWX44OXLUJR...,https://imgcy.trivago.com/partner-images/9c/b1...,bedroom,0.9969,2 Bedroom Flat In Mayfair,2024-03-07,0.820251,None,0.0,0.102159


In [9]:
new_df.to_csv('ladnmark_detection_results.csv', index=False)


In [10]:
new_df

,accommodation_id,longitude,latitude,driving_duration,walking_duration,transit_duration,image_id,image,tag_name,probability,accommodation_name,dt,distance_to_bigben,description,score,response_time
0,37151070,-0.15444,51.49080,16,56,10,BZU37N77BV2F3LHYDM3TMMQOXKZMGOW5B6N6MZYAOCXCF7...,https://imgcy.trivago.com/partner-images/0e/69...,bedroom,0.9999,Passmore Street By Onefinestay,2024-03-07,3.395155,None,0.0,0.273663
1,37151070,-0.15444,51.49080,16,56,10,65OO2YWIMBEYPYI7TULDEBYJ65W5HRUUUF3PMBYLBTS5JE...,https://imgcy.trivago.com/partner-images/f7/5c...,bedroom,0.9954,Passmore Street By Onefinestay,2024-03-07,3.395155,None,0.0,0.338291
2,37151070,-0.15444,51.49080,16,56,10,RZRSSVMYZXVLIQT2VXQAKNO524ZEDJ65NG3JNWOZIYMV3F...,https://imgcy.trivago.com/partner-images/8e/63...,bedroom,0.9999,Passmore Street By Onefinestay,2024-03-07,3.395155,None,0.0,0.327386
3,37151070,-0.15444,51.49080,16,56,10,Y5IZPSMKFH26KUHXOFJRYAEXKZVEL6TCHLDRNUI4LRW64A...,https://imgcy.trivago.com/partner-images/c7/51...,building,0.9996,Passmore Street By Onefinestay,2024-03-07,3.395155,None,0.0,0.542081
4,37151070,-0.15444,51.49080,16,56,10,NUV5T3RYBEGCC4GXLJ3FA3TVCQSQJVPPMGZFPDSNAP6JMY...,https://imgcy.trivago.com/partner-images/6d/2b...,kids-area,0.9716,Passmore Street By Onefinestay,2024-03-07,3.395155,None,0.0,0.615594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76671,25253350,-0.12750,51.50722,4,14,10,OPRDTFXE7GRZWBINQLFHVBCBF67X2ISGDYJOHTMG5AE3QP...,https://imgcy.trivago.com/hotelier-images/73/e...,bedroom,0.9999,2 Bedroom Flat In Mayfair,2024-03-07,0.820251,None,0.0,0.131110
76672,25253350,-0.12750,51.50722,4,14,10,RZFWVMH5TW32S67G74L5EHKDOXAAFKM45O7ERHFZ2TOZ2X...,https://imgcy.trivago.com/partner-images/8e/4b...,bedroom,0.9999,2 Bedroom Flat In Mayfair,2024-03-07,0.820251,None,0.0,0.128994
76673,25253350,-0.12750,51.50722,4,14,10,NGQMN5KJNRMF3TERX7ZG526HV2VWHCACC7DA7KOHNUHDXY...,https://imgcy.trivago.com/partner-images/69/a0...,building,0.9999,2 Bedroom Flat In Mayfair,2024-03-07,0.820251,None,0.0,0.150750
76674,25253350,-0.12750,51.50722,4,14,10,TSY3I2RRLJTKYU67XEIMGE4EJ72TA3APVHLRWX44OXLUJR...,https://imgcy.trivago.com/partner-images/9c/b1...,bedroom,0.9969,2 Bedroom Flat In Mayfair,2024-03-07,0.820251,None,0.0,0.102159
